In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
colunas = [
    'SG_UF',
    'NO_MUNICIPIO',
    'TP_DEPENDENCIA',
    'TP_CATEGORIA_ESCOLA_PRIVADA',
    'TP_SITUACAO_FUNCIONAMENTO',
    'IN_VINCULO_SECRETARIA_EDUCACAO',
    'IN_FORMA_CONT_TERMO_COLABORA',
    'IN_FORMA_CONT_TERMO_FOMENTO',
    'TP_PODER_PUBLICO_PARCERIA',
    'IN_PODER_PUBLICO_PARCERIA',
    'TP_RESPONSAVEL_REGULAMENTACAO',
    'IN_QUADRA_ESPORTES',
    'IN_ACESSIBILIDADE_INEXISTENTE',
    'IN_PARQUE_INFANTIL',
    'IN_PATIO_COBERTO',
    'IN_PATIO_DESCOBERTO',
    'IN_BANHEIRO_EI',
    'IN_SALA_ATENDIMENTO_ESPECIAL',
    'IN_PROF_ALIMENTACAO',
    'IN_ALIMENTACAO',
    'IN_MATERIAL_PED_INFANTIL',
    'IN_MATERIAL_PED_JOGOS',
    'IN_MATERIAL_PED_MUSICAL',
    'IN_MATERIAL_PED_ARTISTICAS',
    'IN_MATERIAL_PED_NENHUM',
    'IN_AREA_VERDE',
    'IN_INF_CRE',
    'IN_INF_PRE',
    'IN_INF',
    'QT_MAT_INF_CRE',
    'QT_MAT_INF_PRE',
    'QT_MAT_INF',
    'QT_MAT_INF_CRE_INT',
    'QT_MAT_INF_PRE_INT',
    'QT_MAT_INF_INT',
    'IN_FUND',
    'QT_MAT_INF',
    'QT_MAT_INF_INT',
    'QT_DOC_INF',
    'QT_TUR_INF'
]

In [4]:
# preparar csc tudo filtrado rio
#df = pd.read_csv('data/microdados_ed_basica_2022.csv', sep=';', encoding='latin1', usecols=colunas)

In [5]:
df = pd.read_csv('data/microdados_ed_basica_2022.csv', sep=';', encoding='latin1')

C:\Users\Gabriel_Zelesco\AppData\Local\Temp\ipykernel_14032\2838323612.py:1: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/microdados_ed_basica_2022.csv', sep=';', encoding='latin1')


In [6]:
df = df.query('(SG_UF == "RJ") & (NO_MUNICIPIO ==  "Rio de Janeiro")')

# Análise

In [ ]:
# Filtrar Conveniadas
'''
'IN_FORMA_CONT_TERMO_FOMENTO' ou 'IN_FORMA_CONT_TERMO_COLABORA'
+
'TP_PODER_PUBLICO_PARCERIA'
'''

In [ ]:
df_conveniadas = (
    df
    .query('(IN_FORMA_CONT_TERMO_FOMENTO == 1) | (IN_FORMA_CONT_TERMO_COLABORA == 1) & (TP_PODER_PUBLICO_PARCERIA == 1) & (IN_INF == 1)')
    .reset_index()
)

In [ ]:
df_conveniadas

In [ ]:
df_publico = (
    df
    .query('(TP_DEPENDENCIA == 3) & (IN_INF == 1)')
    .reset_index()
)

In [ ]:
df_publico

In [ ]:
df_inf_exclusivo = df_publico[df_publico['IN_FUND']==0]

In [ ]:
df_inf_exclusivo

In [ ]:
df_inf_nao_exclusivo = df_publico[df_publico['IN_FUND']==1]

In [ ]:
df_inf_nao_exclusivo

In [ ]:
df_inf_privada= (
    df
    .query('(TP_DEPENDENCIA == 4) & (IN_INF == 1) & (IN_PODER_PUBLICO_PARCERIA == 0)')
    .reset_index()
)

In [ ]:
df_inf_privada

### Contrução do gráfico de 'qualidade' das escolas

In [ ]:
df_infantil = (
    df
    .query('(IN_INF == 1) & (TP_DEPENDENCIA == [3,4])')
    .reset_index()
)

In [ ]:
df_infantil

In [ ]:

adm_list = []
for escola, parceria, segmento in zip(df_infantil['TP_DEPENDENCIA'], df_infantil['IN_PODER_PUBLICO_PARCERIA'], df_infantil['IN_FUND']):
    if escola == 3:
        if segmento == 0:
            adm = 'municipal_exclusivo'
        else:
            adm = 'municipal_nao_exclusivo'
        adm_list.append(adm)
    elif escola == 4:
        if parceria == 0:
            adm = 'privado'
        else:
            adm = 'conveniado'
        adm_list.append(adm)

In [ ]:
df_infantil['adm'] = adm_list

In [ ]:
df_infantil

In [ ]:
df_somatorio = (
    df_infantil
    .groupby(['adm'])
    .agg({'TP_SITUACAO_FUNCIONAMENTO' : 'sum'})
    .reset_index()
)

In [ ]:
df_somatorio

In [ ]:
df_infantil = (
    df_infantil
    .groupby(['adm'])
    .agg({
        'IN_AREA_VERDE' : 'sum',
        'IN_BANHEIRO_EI' : 'sum',
        'IN_PATIO_COBERTO' : 'sum',
        'IN_PATIO_DESCOBERTO' : 'sum',
        'IN_PARQUE_INFANTIL' : 'sum',
        'IN_QUADRA_ESPORTES' : 'sum',
        'IN_SALA_ATENDIMENTO_ESPECIAL' : 'sum',
        'IN_ACESSIBILIDADE_INEXISTENTE' : 'sum',
        'IN_PROF_ALIMENTACAO' : 'sum',
        'IN_ALIMENTACAO' : 'sum',
        'IN_MATERIAL_PED_INFANTIL' : 'sum',
        'IN_MATERIAL_PED_MUSICAL' : 'sum',
        'IN_MATERIAL_PED_JOGOS' : 'sum',
        'IN_MATERIAL_PED_ARTISTICAS' : 'sum',
        'IN_MATERIAL_PED_NENHUM' : 'sum',
        'QT_MAT_INF' : 'sum',
        'QT_MAT_INF_INT' : 'sum',
        'QT_DOC_INF' : 'sum',
        'QT_TUR_INF' : 'sum',
    })
    .reset_index()
)

In [ ]:
df_infantil

In [ ]:
df_infantil = pd.merge(df_infantil, df_somatorio, on='adm')

In [ ]:
df_infantil

In [ ]:
colunas_estrutura = [
    'IN_QUADRA_ESPORTES',
    'IN_ACESSIBILIDADE_INEXISTENTE',
    'IN_PARQUE_INFANTIL',
    'IN_PATIO_COBERTO',
    'IN_PATIO_DESCOBERTO',
    'IN_BANHEIRO_EI',
    'IN_SALA_ATENDIMENTO_ESPECIAL',
    'IN_PROF_ALIMENTACAO',
    'IN_ALIMENTACAO',
    'IN_MATERIAL_PED_INFANTIL',
    'IN_MATERIAL_PED_JOGOS',
    'IN_MATERIAL_PED_MUSICAL',
    'IN_MATERIAL_PED_ARTISTICAS',
    'IN_MATERIAL_PED_NENHUM',
    'IN_AREA_VERDE',
]

In [ ]:
for column in colunas_estrutura:
    df_infantil[f'{column}_pct'] = df_infantil[f'{column}'] / df_infantil['TP_SITUACAO_FUNCIONAMENTO']

In [ ]:
df_infantil

# Gráfico de Qualidade de Estrutura

In [ ]:
colunas_grafico_estrutura = [
    'IN_QUADRA_ESPORTES_pct',
    'IN_ACESSIBILIDADE_INEXISTENTE_pct',
    'IN_PARQUE_INFANTIL_pct',
    'IN_PATIO_COBERTO_pct',
    'IN_PATIO_DESCOBERTO_pct',
    'IN_BANHEIRO_EI_pct',
    'IN_SALA_ATENDIMENTO_ESPECIAL_pct',
    'IN_PROF_ALIMENTACAO_pct',
    'IN_ALIMENTACAO_pct',
    'IN_MATERIAL_PED_INFANTIL_pct',
    'IN_MATERIAL_PED_JOGOS_pct',
    'IN_MATERIAL_PED_MUSICAL_pct',
    'IN_MATERIAL_PED_ARTISTICAS_pct',
    'IN_MATERIAL_PED_NENHUM_pct',
    'IN_AREA_VERDE_pct',
]

In [ ]:
df_long = pd.melt(df_infantil, id_vars='adm', value_vars=colunas_grafico_estrutura)

In [ ]:
df_long 

In [ ]:
ax = sns.scatterplot(
    data=df_long,
    x='variable',
    y='value',
    hue='adm'
)

ax.set_xticklabels(df_long.variable.unique(), rotation=90)
ax.legend(bbox_to_anchor=(1.1, 1.05))
ax.grid()

In [ ]:
f, ax = plt.subplots(figsize=(8, 10))
sns.barplot(data=df_long, x="value", y="variable", hue="adm", ax=ax, palette='inferno')
ax.legend(bbox_to_anchor=(1.4, 1.01))
ax.grid()

## Graficos de Qualidade

In [ ]:
colunas_qualidade = [
    'adm',
    'QT_MAT_INF',
    'QT_MAT_INF_INT',
    'QT_DOC_INF',
    'QT_TUR_INF',
    'TP_SITUACAO_FUNCIONAMENTO'
]

In [ ]:
df_qualidade = df_infantil[colunas_qualidade]

In [ ]:
df_qualidade

In [ ]:
df_qualidade = df_qualidade.assign(
    matriculas_por_turmas = df_qualidade['QT_MAT_INF'] / df_qualidade['QT_TUR_INF'],
    mat_integral = df_qualidade['QT_MAT_INF_INT'] / df_qualidade['QT_MAT_INF'],
    matriculas_por_docentes = df_qualidade['QT_MAT_INF'] / df_qualidade['QT_DOC_INF'],
    turmas_por_docentes = df_qualidade['QT_TUR_INF'] / df_qualidade['QT_DOC_INF']
)

In [ ]:
df_qualidade

Exportação da planilha principal sobre qualidade

In [ ]:
colunas_export = [
    'adm',
    'matriculas_por_turmas',
    'mat_integral',
    'matriculas_por_docentes',
    'turmas_por_docentes',
]

In [ ]:
df_long_export= pd.melt(df_qualidade, id_vars='adm', value_vars=colunas_export)

In [ ]:
df_long_export

In [ ]:
df_long_export.to_excel('infantil_escolas_qualidade.xlsx', index=False)

## Grafico 1: Matriculas vs Turmas

In [ ]:
colunas_grafico1 = [
    'adm',
    'matriculas_por_turmas',
]

In [ ]:
df_long1 = pd.melt(df_qualidade, id_vars='adm', value_vars=colunas_grafico1)

In [ ]:
df_long1

In [ ]:
# Matriculas vs Turmas
f, ax = plt.subplots(figsize=(4, 4))
sns.barplot(data=df_long1,
            x="variable",
            y="value",
            hue="adm",
            ax=ax,
            palette='inferno')

ax.legend(bbox_to_anchor=(1.4, 1.01))
ax.set(title='Matriculas por Turma',
       ylabel='qtd matriculas',
       xlabel=None)

ax.grid()

## Grafico 2: % de matriculas em tempo integral

In [ ]:
colunas_grafico2 = [
    'adm',
    'mat_integral',
]

In [ ]:
df_long2 = pd.melt(df_qualidade, id_vars='adm', value_vars=colunas_grafico2)

In [ ]:
df_long2 

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
sns.barplot(data=df_long2,
            x="variable",
            y="value",
            hue="adm",
            ax=ax,
            palette='inferno')

ax.legend(bbox_to_anchor=(2, 1.01))
ax.set(title='% de matriculas em tempo integral',
       ylabel='pct matriculas',
       xlabel=None)

ax.grid()

## Grafico 3: Matriculas por docentes

In [ ]:
colunas_grafico3 = [
    'adm',
    'matriculas_por_docentes',
]

In [ ]:
df_long3 = pd.melt(df_qualidade, id_vars='adm', value_vars=colunas_grafico3)

In [ ]:
df_long3

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
sns.barplot(data=df_long3,
            x="variable",
            y="value",
            hue="adm",
            ax=ax,
            palette='inferno')

ax.legend(bbox_to_anchor=(1.4, 1.01))
ax.set(title='Matriculas por Docente',
       ylabel='qtd matriculas',
       xlabel=None)

ax.grid()

## Grafico 4: Turmas por Docente

In [ ]:
colunas_grafico4 = [
    'adm',
    'turmas_por_docentes',
]

In [ ]:
df_long4 = pd.melt(df_qualidade, id_vars='adm', value_vars=colunas_grafico4)

In [ ]:
df_long4

In [ ]:
f, ax = plt.subplots(figsize=(4, 4))
sns.barplot(data=df_long4,
            x="variable",
            y="value",
            hue="adm",
            ax=ax,
            palette='inferno')

ax.legend(bbox_to_anchor=(1.4, 1.01))
ax.set(title='Turmas por Docente',
       ylabel='qtd turmas',
       xlabel=None)

ax.grid()